In [0]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

import pandas as pd
import string
import re
import nltk
from nltk.stem import WordNetLemmatizer 

In [11]:
from google.colab import files
uploaded = files.upload()

Saving model_data (1).csv to model_data (1).csv


In [0]:
df = pd.read_csv('model_data (1).csv')


In [13]:
df.head()

,subreddits,post_title,post_content
0,Home,Contract signed and waiting,\nNJ homeowner. \n\nI signed a contract to hav...
1,Home,How to repair/re-plaster old ceiling? It had s...,NaN
2,Home,Can I make my matte black countertops shiny?!,NaN
3,Home,Did you lock the door before leaving?,A lot of people here seem to forget whether t...
4,Home,How do I replace my air filter on this thing???,NaN


In [0]:
df = df.sample(frac=1).reset_index(drop=True)

In [15]:
df.fillna('0')

,subreddits,post_title,post_content
0,Coronavirus,The Pandemic’s health side effects are growing,0
1,AskReddit,You mash up your two favorite games together i...,0
2,movies,Larry Kramer Dead: Playwright and AIDS Activis...,0
3,OldSchoolCool,"Commercial for ""Boglins,"" creepy puppets from ...",0
4,pcmasterrace,Literally my parents,0
...,...,...,...
9959,Showerthoughts,If you watch a non native speaker speak your n...,For example: A german person speaking english ...
9960,discordapp,Discord iOS keeps crashing,"When I open discord, it’s open for a split sec..."
9961,ChoosingBeggars,"Football clue advising in the job section, bri...",0
9962,oddlysatisfying,Pickled ginger brine evaporated and left behin...,0


In [0]:
df["content"] = df["post_title"].astype(str) + df["post_content"].astype(str)

In [17]:
df.head()

,subreddits,post_title,post_content,content
0,Coronavirus,The Pandemic’s health side effects are growing,NaN,The Pandemic’s health side effects are growingnan
1,AskReddit,You mash up your two favorite games together i...,NaN,You mash up your two favorite games together i...
2,movies,Larry Kramer Dead: Playwright and AIDS Activis...,NaN,Larry Kramer Dead: Playwright and AIDS Activis...
3,OldSchoolCool,"Commercial for ""Boglins,"" creepy puppets from ...",NaN,"Commercial for ""Boglins,"" creepy puppets from ..."
4,pcmasterrace,Literally my parents,NaN,Literally my parentsnan


In [0]:
df=df.drop(['post_title', 'post_content'], axis=1)

In [0]:
df.columns = ['subreddit', 'content']

In [20]:
X = df['content']
y = df['subreddit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(7971,)
(7971,)
(1993,)
(1993,)


In [0]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('classifier', RandomForestClassifier()),
])

In [22]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_patt...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None

In [23]:
predictions = pipeline.predict(X_test)
print(classification_report(y_test,predictions))

                     precision    recall  f1-score   support

      AmItheAsshole       0.51      0.97      0.67        35
                Amd       0.82      0.70      0.76        47
             AskMen       0.30      0.19      0.23        43
          AskReddit       0.24      0.47      0.32        40
           AskWomen       0.38      0.32      0.35        34
    ChoosingBeggars       0.50      0.29      0.36        42
        Coronavirus       0.39      0.71      0.50        38
                DnD       0.85      0.77      0.80        43
               Home       0.56      0.36      0.43        42
               IAmA       0.74      0.95      0.83        42
       IdiotsInCars       0.31      0.32      0.31        41
  NoStupidQuestions       0.46      0.23      0.31        52
      OldSchoolCool       0.63      0.42      0.51        40
       OutOfTheLoop       0.51      0.95      0.66        43
             Piracy       0.66      0.68      0.67        40
     Showerthoughts    

In [0]:
def get_predictions(post, num_answers=5):
  """ takes a potential post and returns the top options """

  preds = pd.Series(pipeline.predict_proba(post)[0])

  preds.index = pipeline.classes_

  preds = preds.sort_values(ascending=False)

  return preds[:5]

In [0]:
from pickle import dump
dump(pipeline, open('reddit_model_nc.pkl', 'wb'))

In [0]:
nba_post = [ """
               LeBron James and Kobe Bryant and both great NBA players and we should stop comparing them.
                """]

In [26]:
get_predictions(nba_post)

nba              0.66
OldSchoolCool    0.05
gaming           0.05
politics         0.04
news             0.04
dtype: float64

In [0]:
old